# Training Siamese

In [29]:
import torch
import pickle
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
from siamese import Siamese
import time
import numpy as np
from collections import deque
import os
import pandas as pd
from dataset import CarDataset
import sys

cuda = False
way = 20
times = 400
workers = 4
bacth_size = 128
lr = 0.00006
show_every = 10
save_every = 100
test_every = 100
max_iter = 50_000
model_path = ""
gpu_ids = "0"


data_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])


## Define dataset

In [30]:
dataset = pd.read_csv('../patches/gt_car_patches_annotations.csv', delimiter=',')

train = dataset[(dataset.SEQ == 'S01') | (dataset.SEQ == 'S04')]
test = dataset[(dataset.SEQ == 'S03')]

print(f"Train size: {train.shape[0]} Test size {test.shape[0]}")

Train size: 38074 Test size 6174


In [31]:
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_ids
print("use gpu:", gpu_ids, "to train.")

trainSet = CarDataset(train, transform=data_transforms)
testSet = CarDataset(test, transform=data_transforms)

testLoader = DataLoader(testSet, batch_size=way, shuffle=False, num_workers=workers)
trainLoader = DataLoader(trainSet, batch_size=bacth_size, shuffle=False, num_workers=workers)

loss_fn = torch.nn.BCEWithLogitsLoss(size_average=True)
net = Siamese()

# multi gpu
if len(gpu_ids.split(",")) > 1:
    net = torch.nn.DataParallel(net)

if cuda:
    net.cuda()

net.train()

optimizer = torch.optim.Adam(net.parameters(), lr = lr)
optimizer.zero_grad()

train_loss = []
loss_val = 0
time_start = time.time()
queue = deque(maxlen=20)

for batch_id, (img1, img2, label) in enumerate(trainLoader, 1):
    if batch_id > max_iter:
        break
    if cuda:
        img1, img2, label = Variable(img1.cuda()), Variable(img2.cuda()), Variable(label.cuda())
    else:
        img1, img2, label = Variable(img1), Variable(img2), Variable(label)

    optimizer.zero_grad()
    output = net.forward(img1, img2)
    loss = loss_fn(output, label)
    loss_val += loss.item()
    loss.backward()
    optimizer.step()

    if batch_id % show_every == 0 :
        print('[%d]\tloss:\t%.5f\ttime lapsed:\t%.2f s'%(batch_id, loss_val/show_every, time.time() - time_start))
        loss_val = 0
        time_start = time.time()
    if batch_id % save_every == 0:
        torch.save(net.state_dict(), model_path + '/model-inter-' + str(batch_id+1) + ".pt")

    if batch_id % test_every == 0:
        right, error = 0, 0
        for _, (test1, test2) in enumerate(testLoader, 1):
            if cuda:
                test1, test2 = test1.cuda(), test2.cuda()

            test1, test2 = Variable(test1), Variable(test2)
            output = net.forward(test1, test2).data.cpu().numpy()
            pred = np.argmax(output)
            if pred == 0:
                right += 1
            else: error += 1

        print('*'*70)
        print('[%d]\tTest set\tcorrect:\t%d\terror:\t%d\tprecision:\t%f'%(batch_id, right, error, right*1.0/(right+error)))
        print('*'*70)
        queue.append(right*1.0/(right+error))

    sys.stdout.flush()
    train_loss.append(loss_val)
#  learning_rate = learning_rate * 0.95

with open('train_loss', 'wb') as f:
    pickle.dump(train_loss, f)

acc = 0.0
for d in queue:
    acc += d
print("#"*70)

use gpu: 0 to train.


/Users/kevinmartinfernandez/opt/anaconda3/envs/M6_Video/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x1000 and 9216x4096)